## Task1 -Part 2
### Predict sentiment score for test data 

### Import required libraries

In [1]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

from keras.layers import Flatten
from keras import regularizers

Using TensorFlow backend.


### Load test data

In [2]:
with open('Microblog_Trainingdata.json') as data_file:    
    Microblog_Trainingdata = json.load(data_file)

for i in range(0,len(Microblog_Trainingdata)):
    Microblog_Trainingdata[i]['spans']=Microblog_Trainingdata[i]['spans'][0]
    Microblog_Trainingdata[i]['sentiment score']=float(Microblog_Trainingdata[i]['sentiment score'])
#pprint(Microblog_Trainingdata)
with open('Microblog_Trialdata.json') as data_file:   
    Microblog_Trialdata = json.load(data_file)
    

#pprint(data)
for i in range(0,len(Microblog_Trialdata)):
    Microblog_Trialdata[i]['spans']=Microblog_Trialdata[i]['spans'][0]
    Microblog_Trialdata[i]['sentiment score']=float(Microblog_Trialdata[i]['sentiment score'])

In [3]:
with open('Microblogs_Testdata.json') as data_file:   
    Microblogs_Testdata = json.load(data_file)

In [4]:
for i in Microblog_Trialdata:
    Microblog_Trainingdata.append(i)

### Convert data to lowercase format

In [5]:
def lower_keys(x):
    if isinstance(x, list):
        return [lower_keys(v) for v in x]
    elif isinstance(x, dict):
        return dict((k.lower(), v.lower() if isinstance(v, str) else v) for k, v in x.items())
    else:
        return x
Microblog_Trainingdata=lower_keys(Microblog_Trainingdata)
Microblogs_Testdata=lower_keys(Microblogs_Testdata)

### Helper functions
  1. Get stopwords from English and remove them from input data
  2. Remove punctuations and non-ascii characters from input data

In [6]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [7]:
import string
def stringClean( inputString ) :
    return inputString.translate(str.maketrans('','',string.punctuation)).encode("ascii", errors="ignore").decode()

In [8]:
def removeStopWords(inputString):
    splitString = inputString.split()
    for word in stop:
        for s in splitString:
            if s == word:
                splitString.remove(s)
    return " ".join(splitString)

In [9]:
maxlen = 0

X_train = []
X_trial=[]
Y_trial=[]
X_test = [(removeStopWords(stringClean(str(Microblog['spans'])))) for Microblog in Microblogs_Testdata]
Y_train = []
Y_test = []
output=[]
for Microblog in Microblog_Trainingdata :
    X_train.append(removeStopWords(stringClean(Microblog['spans'])))
    Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
    output.append(Microblog['sentiment score'])
    if maxlen < len(Microblog['spans'].split()) :
        maxlen = len(Microblog['spans'].split())

#for Microblog in Microblog_Trialdata :
 #   X_train.append(Microblog['spans'])
  #  Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
   # if maxlen < len(Microblog['spans'].split()) :
    #    maxlen = len(Microblog['spans'].split())


print(len(X_train)+len(X_test), len(Y_train))

2510 1710


### Tokenize the data

In [10]:
tokenizer = Tokenizer(num_words=maxlen*20)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen)
data.shape

(1710, 25)

### Build the model

In [11]:
model = Sequential()
model.add(Embedding(maxlen*20, 32, input_length=maxlen))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='leakyrelu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.15))
#, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)
model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 32)            16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 24,457
Trainable params: 24,457
Non-trainable params: 0
_________________________________________________________________


### Train the model with validation split 0.2 and 15 epochs

In [12]:
history = model.fit(data, np.array(Y_train), validation_split=0.2, epochs=15,verbose = 1)

Train on 1368 samples, validate on 342 samples
Epoch 1/15
1368/1368 [==============================] - 3s - loss: 1.0143 - acc: 0.4613 - val_loss: 0.6568 - val_acc: 0.6462
Epoch 2/15
1368/1368 [==============================] - 1s - loss: 0.6575 - acc: 0.6360 - val_loss: 0.6385 - val_acc: 0.6520
Epoch 3/15
1368/1368 [==============================] - 1s - loss: 0.6246 - acc: 0.6827 - val_loss: 0.6074 - val_acc: 0.6813
Epoch 4/15
1368/1368 [==============================] - 1s - loss: 0.5735 - acc: 0.7412 - val_loss: 0.5548 - val_acc: 0.7222
Epoch 5/15
1368/1368 [==============================] - 1s - loss: 0.4881 - acc: 0.7975 - val_loss: 0.4807 - val_acc: 0.7749
Epoch 6/15
1368/1368 [==============================] - 1s - loss: 0.4350 - acc: 0.8268 - val_loss: 0.4882 - val_acc: 0.7807
Epoch 7/15
1368/1368 [==============================] - 1s - loss: 0.3987 - acc: 0.8194 - val_loss: 0.5613 - val_acc: 0.7924
Epoch 8/15
1368/1368 [==============================] - 1s - loss: 0.3509 - ac

### Tokenize the testing data and predict sentiment score

In [13]:
sequences = tokenizer.texts_to_sequences(np.array(X_test))
data = pad_sequences(sequences, maxlen)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(Microblogs_Testdata[i]['spans'],"%.3f" % predictions[i])

consumers keep cautious stance 0.054
close $ctrp @46.16 from 43.55 entry +6%;$;+6% 0.225
every reason to be bullish 0.990
$panw need anoth $1 to all time high -0.418
long setup 0.988
will be a solid long term investmen;$;#passive;$;also added $sbux today;$; #income 0.990
recall 0.617
marriott wins new starwood deal by raising cash portion of bid 0.991
time to embrace dividend growth 0.980
bullshit! all #applestores are empty in #china;$;$ms must be preparing to liquidate $aapl 0.428
5-star analyst robert derrington from wunderlich reiterated a buy 0.981
excited;$;big +value;$;hoping 0.740
oil to break out;$;adding chevron 0.970
unusual call activity 0.992
china's tourist bonanza isn't going anywhere 0.559
0.15 0.740
not guaranteed 0.740
potential continuation uptrend 0.984
longed 0.740
absolute garbage still up;$;stores totally empty;$;stock mispriced 0.747
buy the dip' 0.883
weakest in todays dt watchlist 0.942
we see more reasons to be cautious -0.433
breaking out 0.917
$bbry smack d

top stocks in 2015 are ready to lead again in 2016 0.992
not getting in;$;oil is falling more -0.040
competing bids to emerge, and will permit the board to explore other value-enhancing alternatives, all with the objective of maximizing 0.740
the resurgence in stock price today might mean that the company may emerge stronger than ever 0.959
supermarket stocks rally as tesco plans to axe non-core uk assets 0.989
long;$;buying point 0.975
bear raid -0.474
#trading weak vs. $spy & $qqq 0.410
short -0.126
expects strong results 0.969
$calx +40 0.740
one of the more obvious opportunities this year 0.308
holding up relatively well compared to the broad market 0.980
i would rather be a buyer ugaz at these levels then trying to sell -0.334
will go up much higher after this drop. 0.729
i dumped $pypl and $prlb 0.740
the real deal here...holding 0.978
large players are selling 0.359
yahoo extends bidding deadline as interest grows 0.989
could offer support revisit to jump on long bus? 0.992
new 

### Create a dataframe of results and create a csv file

In [14]:
df=pd.DataFrame.from_dict(Microblogs_Testdata)

In [15]:
df['Predicted Sentiment Score']=predictions
df.head()

,cashtag,id,source,spans,Predicted Sentiment Score
0,$cost,709723193125175300,twitter,consumers keep cautious stance,0.054084
1,$ctrp,719529094652698600,twitter,close $ctrp @46.16 from 43.55 entry +6%;$;+6%,0.224955
2,$intc,39048670,stocktwits,every reason to be bullish,0.989760
3,$panw,37048093,stocktwits,$panw need anoth $1 to all time high,-0.417503
4,$jrcc,6207860,stocktwits,long setup,0.988097


In [16]:
df.to_csv('TestData.csv',index=False)